# ETF Categorization

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import justetf_scraping
from justetf_scraping import classes_categories_etf

C:\Users\Alberto Gerri\AppData\Local\Temp\ipykernel_5480\3598951620.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# scraping sul sito di tutte le classes/category/subcategory
url = 'https://www.justetf.com/find-etf.html?'
class_scraping=justetf_scraping.get_classes_categories(url)

In [4]:
class_scraping['Equity']['Theme']

['Ageing_Population',
 'Alternative_Investments',
 'Artificial_Intelligence',
 'Battery_Technology',
 'Biotech',
 'Blockchain',
 'Buyback',
 'Circular_Economy',
 'Clean_Energy',
 'Climate_Change',
 'Cloud_Technology',
 'Cyber_Security',
 'Digitalisation',
 'Ecommerce',
 'E_Sports',
 'Fintech',
 'Mobility',
 'Food',
 'Gender_Equality',
 'Goldminers',
 'Hydrogen',
 'Innovation',
 'IPO',
 'Islamic',
 'Luxury',
 'Metaverse',
 'Defense',
 'Millenials',
 'MLP',
 'Moat',
 'Multi_Asset',
 'Photonics',
 'Private_Equity',
 'Robotics',
 'Semiconductors',
 'Space',
 'Uranium',
 'Volatility',
 'Water',
 'Wood',
 'none']

In [5]:
# definisco le classi su cui iterare  (sarà unica colonna "class") 
classes=list(class_scraping.keys())
# definisco le categorie su cui iterare (saranno colonne diverse)
# definisco le sotto-categorie su cui iterare (saranno i valori che popoleranno le colonne "categories")
categories=[]
subcategories=[]
for class_ in classes:
    print(class_)
    categories_=list(class_scraping[class_].keys())
    categories=categories + categories_
    for category_ in categories_:
        subcategories_=class_scraping[class_][category_]
        subcategories=subcategories+subcategories_

categories=list(set(categories))
subcategories=list(set(subcategories))

matrice_semplice = justetf_scraping.load_overview()
empty_df=pd.DataFrame(columns=['Class']+categories)
matrice_semplice=pd.concat([matrice_semplice,empty_df],axis='columns')
# definisco cosa voglio raccogliere: una matrice in cui metto tutti gli isin NON ripetuti + classi (sono 3, diverse), categorie e sotto-categorie
# Quindi matrice data da "overview" + 3 colonne. 
# matrice=pd.DataFrame(columns=['Class']+categories)

# Gestione eccezioni: 1) se non trova nulla nella ricerca, next ; 2) se l'isin è già presente nella matrice devo compilare le colonne non compilate. 
# Se alcune colonne sono già compilate con dei valori, da capire
# Ciclo For
iterazioni=0
for class_ in class_scraping:
    for category_ in class_scraping[class_]:
        memoria_subcategory=[]
        for subcategory_ in class_scraping[class_][category_]:
            iterazioni+=1


Equity
Bonds
Precious Metals
Commodities
Cryptocurrencies
Real Estate
Money Market


In [6]:

it=0
it_test='no_test'
# crea file vuoto log
with open('output\\logs.txt', "w") as file:
    file.write('LOGS \n')
for class_ in class_scraping:
    if it==it_test:
        break
    for category_ in class_scraping[class_]:
        memoria_subcategory_isin=[]
        memoria_subcategory=[]
        if it==it_test:
            break
        for subcategory_ in class_scraping[class_][category_]:
            # cerchiamo per classe, category,subcategory
            it+=1
            #controllo test
            if it==it_test:
                break

            print(f'[{it}/{iterazioni}] Scraping per -> classe: {class_}, categoria: {category_}, sub-categoria: {subcategory_}')
            parameters={
                'asset': classes_categories_etf.ASSETCLASS_ENG[class_],
                'bondRating':       subcategory_.replace(" ","%2B") if classes_categories_etf.CATEGORIES_API_ENG[category_] == 'bondRating' else None,
                'bondStrategy':     subcategory_.replace(" ","%2B") if classes_categories_etf.CATEGORIES_API_ENG[category_] == 'bondStrategy' else None,
                'bondType':         subcategory_.replace(" ","%2B") if classes_categories_etf.CATEGORIES_API_ENG[category_] == 'bondType' else None,
                'ctype':            subcategory_.replace(" ","%2B") if classes_categories_etf.CATEGORIES_API_ENG[category_] == 'ctype' else None,
                'country':          subcategory_.replace(" ","%2B") if classes_categories_etf.CATEGORIES_API_ENG[category_] == 'country' else None,
                'currency':         subcategory_.replace(" ","%2B") if classes_categories_etf.CATEGORIES_API_ENG[category_] == 'currency' else None,
                'equityStrategy':   subcategory_.replace(" ","%2B") if classes_categories_etf.CATEGORIES_API_ENG[category_] == 'equityStrategy'else None,
                'bm':               subcategory_.replace(" ","%2B") if classes_categories_etf.CATEGORIES_API_ENG[category_] == 'bm' else None,
                'region':           subcategory_.replace(" ","%2B") if classes_categories_etf.CATEGORIES_API_ENG[category_] == 'region' else None,
                'sector':           subcategory_.replace(" ","%2B") if classes_categories_etf.CATEGORIES_API_ENG[category_] == 'sector' else None,
                'cf':               subcategory_.replace(" ","%2B") if classes_categories_etf.CATEGORIES_API_ENG[category_] == 'cf' else None,
                'theme':            subcategory_.replace(" ","%2B") if classes_categories_etf.CATEGORIES_API_ENG[category_] == 'theme' else None
            }
            # gestisco eccezione 1
            try:
                df=justetf_scraping.load_overview(**parameters)
            except:
                continue
            # check se isin già apparso per la stessa categoria
            memoria_subcategory_isin+=list(df.index)
            memoria_subcategory+=[subcategory_]*len(list(df.index))

            # aggiungo classe e categoria
            empty_df=pd.DataFrame(columns=['Class']+categories)
            df=pd.concat([df,empty_df],axis='columns')
            df.loc[:,'Class']=class_
            df.loc[:,category_]=subcategory_
            
            # aggiungo df alla matrice stando a attento se isin c'è già nella matrice
            # merge outer on isin? da fare qualche prova
            if it==1:
                matrice=df
            else:
                # Unione dei DataFrame in base all'indice 'isin'
                # trovo isin già presenti
                isin_comuni=matrice.index.intersection(df.index)
                if not isin_comuni.empty:
                    for i in isin_comuni:
                        matrice.loc[i,category_]=subcategory_
                isin_nuovi=df.index.difference(matrice.index)
                if not isin_nuovi.empty:
                    matrice=pd.concat([matrice,df.loc[isin_nuovi,:]],axis='rows')

        
        # check se c'è qualche isin doppio per categoria e scrivilo in log
        subcat_double={'isin': memoria_subcategory_isin, 'Colonna2': memoria_subcategory}
        subcat_double_df = pd.DataFrame(subcat_double)
        subcat_double_res = subcat_double_df[subcat_double_df.duplicated(subset=['isin'], keep=False)]
        if not subcat_double_res.empty:
            with open('output\\logs.txt', "a") as file:
                for isin_unique in subcat_double_res['isin'].unique():
                    subcat_double_res_string=' - '.join(list(subcat_double_res.loc[subcat_double_res['isin']==isin_unique,'Colonna2']))
                    file.write(f'Isin: {isin_unique}, category: {category_}, presente con: {subcat_double_res_string}')
                    file.write("\n")
                    

[1/199] Scraping per -> classe: Equity, categoria: Region, sub-categoria: Africa
[2/199] Scraping per -> classe: Equity, categoria: Region, sub-categoria: Asia Pacific
[3/199] Scraping per -> classe: Equity, categoria: Region, sub-categoria: Eastern Europe
[4/199] Scraping per -> classe: Equity, categoria: Region, sub-categoria: Emerging Markets
[5/199] Scraping per -> classe: Equity, categoria: Region, sub-categoria: Europe
[6/199] Scraping per -> classe: Equity, categoria: Region, sub-categoria: Latin America
[7/199] Scraping per -> classe: Equity, categoria: Region, sub-categoria: North America
[8/199] Scraping per -> classe: Equity, categoria: Region, sub-categoria: World
[9/199] Scraping per -> classe: Equity, categoria: Country, sub-categoria: AU
[10/199] Scraping per -> classe: Equity, categoria: Country, sub-categoria: AT
[11/199] Scraping per -> classe: Equity, categoria: Country, sub-categoria: BR
[12/199] Scraping per -> classe: Equity, categoria: Country, sub-categoria: BG


C:\Users\Alberto Gerri\Documents\00_ex_Onedrive_Poli\coding\justetf-scraping-1\justetf_scraping\overview.py:490: UserWarning: Received overview columns are not as expected.
	Missing columns: 'isin', 'wkn', 'ticker', 'valorNumber', 'name', 'inceptionDate', 'strategy', 'domicileCountry', 'fundCurrency', 'hasSecuritiesLending', 'distributionPolicy', 'ter', 'replicationMethod', 'fundSize', 'sustainable', 'numberOfHoldings', 'ytdReturnCUR', 'weekReturnCUR', 'monthReturnCUR', 'threeMonthReturnCUR', 'sixMonthReturnCUR', 'yearReturnCUR', 'threeYearReturnCUR', 'fiveYearReturnCUR', 'yearReturn1CUR', 'yearReturn2CUR', 'yearReturn3CUR', 'yearReturn4CUR', 'currentDividendYield', 'yearDividendYield', 'yearVolatilityCUR', 'threeYearVolatilityCUR', 'fiveYearVolatilityCUR', 'yearReturnPerRiskCUR', 'threeYearReturnPerRiskCUR', 'fiveYearReturnPerRiskCUR', 'maxDrawdownCUR', 'yearMaxDrawdownCUR', 'threeYearMaxDrawdownCUR', 'fiveYearMaxDrawdownCUR'.
  warnings.warn(message)


[156/199] Scraping per -> classe: Bonds, categoria: Maturity, sub-categoria: 3-5
[157/199] Scraping per -> classe: Bonds, categoria: Maturity, sub-categoria: 5-7
[158/199] Scraping per -> classe: Bonds, categoria: Maturity, sub-categoria: 7-10
[159/199] Scraping per -> classe: Bonds, categoria: Maturity, sub-categoria: All maturities
[160/199] Scraping per -> classe: Bonds, categoria: Bond Rating, sub-categoria: AAA
[161/199] Scraping per -> classe: Bonds, categoria: Bond Rating, sub-categoria: InvestmentGrade
[162/199] Scraping per -> classe: Bonds, categoria: Bond Rating, sub-categoria: All
[163/199] Scraping per -> classe: Bonds, categoria: Bond Rating, sub-categoria: SubInvestmentGrade
[164/199] Scraping per -> classe: Bonds, categoria: Bond Strategy, sub-categoria: Credit
[165/199] Scraping per -> classe: Bonds, categoria: Bond Strategy, sub-categoria: Maturity bonds
[166/199] Scraping per -> classe: Bonds, categoria: Bond Strategy, sub-categoria: Social / Environmental
[167/199] 

In [7]:
#check. La matrice finale dovrebbe avere stesso numero di righe della matrice trovata con un semplce overview
if matrice_semplice.shape[0]==matrice.shape[0]:
    print('Numero righe combacia')
else:
    print('Numero righe non combacia')
    isin_non_cat=matrice_semplice.index.difference(matrice.index)
    matrice=pd.concat([matrice,matrice_semplice.loc[isin_non_cat,:]],axis='rows')

Numero righe non combacia


In [8]:
# Salviamo matrice in excel 
matrice.to_excel('output\\matrice_etf_categorizzati.xlsx')